# SOCCA!

In [16]:
ls ./src

dratings.txt         raw_matches.csv      thepowerrank.txt
eloratings.txt       raw_winners.csv      wc2018qualified.csv
fifaelo.txt          simulateworldcup.py  wc2018schedule.csv
fifarank.txt         soccerway.txt
matches.csv          team_renames.csv


In [17]:
raw_match_file = './src/raw_matches.csv'
raw_win_file = './src/raw_winners.csv'
rename_file = './src/team_renames.csv'

In [18]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [19]:
def apply_renames(column):
    with open(rename_file) as renames_file:
        renames = dict(l.strip().split(',') for l in renames_file.readlines() if l.strip())

        def renamer(team):
            return renames.get(team, team)

    return column.map(renamer)

In [27]:
def get_matches(with_team_stats=False, duplicate_with_reversed=False,
                exclude_ties=False):
    matches = pd.read_csv(raw_match_file)
    for column in ('team1', 'team2'):
        matches[column] = apply_renames(matches[column])

    if duplicate_with_reversed:
        id_offset = len(matches)

        matches2 = matches.copy()
        matches2.rename(columns={'team1': 'team2',
                                 'team2': 'team1',
                                 'score1': 'score2',
                                 'score2': 'score1'},
                        inplace=True)
        matches2.index = matches2.index.map(lambda x: x + id_offset)

        matches = pd.concat((matches, matches2))

    def winner_from_score_diff(x):
        if x > 0:
            return 1
        elif x < 0:
            return 2
        else:
            return 0

    matches['score_diff'] = matches['score1'] - matches['score2']
    matches['winner'] = matches['score_diff']
    matches['winner'] = matches['winner'].map(winner_from_score_diff)

    if exclude_ties:
        matches = matches[matches['winner'] != 0]

    if with_team_stats:
        stats = get_team_stats()

        matches = matches.join(stats, on='team1').join(stats, on='team2', rsuffix='_2')

    return matches

In [38]:
match = get_matches()
match.shape

(767, 8)

In [39]:
match.head()

,id,year,team1,score1,score2,team2,score_diff,winner
0,0,1950,Brazil,4,0,Mexico,4,1
1,1,1950,Yugoslavia,3,0,Switzerland,3,1
2,2,1950,Brazil,2,2,Switzerland,0,0
3,3,1950,Yugoslavia,4,1,Mexico,3,1
4,4,1950,Brazil,2,0,Yugoslavia,2,1


In [29]:
def get_winners():
    winners = pd.read_csv(raw_win_file)
    winners.team = apply_renames(winners.team)

    return winners

In [35]:
win = get_winners()

In [37]:
win.head()

,year,position,team
0,1950,1,Uruguay
1,1950,2,Brazil
2,1950,3,Sweden
3,1950,4,Spain
4,1954,1,Germany


In [36]:
win.shape

(64, 3)

In [40]:
def get_team_stats():
    winners = get_winners()
    matches = get_matches()

    teams = set(matches.team1.unique()).union(matches.team2.unique())

    stats = pd.DataFrame(list(teams), columns=['team'])

    stats = stats.set_index('team')

    for team in teams:
        team_matches = matches[(matches.team1 == team) |
                               (matches.team2 == team)]
        stats.loc[team, 'matches_played'] = len(team_matches)

        wins1 = team_matches[(team_matches.team1 == team) &
                             (team_matches.score1 > team_matches.score2)]
        wins2 = team_matches[(team_matches.team2 == team) &
                             (team_matches.score2 > team_matches.score1)]

        stats.loc[team, 'matches_won'] = len(wins1) + len(wins2)

        stats.loc[team, 'years_played'] = len(team_matches.year.unique())

        team_podiums = winners[winners.team == team]
        to_score = lambda position: 2 ** (5 - position)  # better position -> more score, exponential
        stats.loc[team, 'podium_score'] = team_podiums.position.map(to_score).sum()

        stats.loc[team, 'cups_won'] = len(team_podiums[team_podiums.position == 1])

    stats['matches_won_percent'] = stats['matches_won'] / stats['matches_played'] * 100.0
    stats['podium_score_yearly'] = stats['podium_score'] / stats['years_played']
    stats['cups_won_yearly'] = stats['cups_won'] / stats['years_played']

    return stats

In [42]:
get_team_stats()

,matches_played,matches_won,years_played,podium_score,cups_won,matches_won_percent,podium_score_yearly,cups_won_yearly
team,,,,,,,,
United States,28.0,6.0,8.0,False,0.0,21.428571,0,0.000000
Yugoslavia,34.0,14.0,8.0,2,0.0,41.176471,0.25,0.000000
South Korea,31.0,5.0,9.0,2,0.0,16.129032,0.222222,0.000000
Togo,3.0,0.0,1.0,False,0.0,0.000000,0,0.000000
Iraq,3.0,0.0,1.0,False,0.0,0.000000,0,0.000000
Wales,5.0,1.0,1.0,False,0.0,20.000000,0,0.000000
North Korea,7.0,1.0,2.0,False,0.0,14.285714,0,0.000000
South Africa,9.0,2.0,3.0,False,0.0,22.222222,0,0.000000
Uruguay,46.0,16.0,11.0,22,1.0,34.782609,2,0.090909


In [48]:
matches = get_matches(with_team_stats=True,
                      duplicate_with_reversed=True,
                      exclude_ties=True)
        
matches.head()

,id,score1,score2,team1,team2,year,score_diff,winner,matches_played,matches_won,...,podium_score_yearly,cups_won_yearly,matches_played_2,matches_won_2,years_played_2,podium_score_2,cups_won_2,matches_won_percent_2,podium_score_yearly_2,cups_won_yearly_2
0,0,4,0,Brazil,Mexico,1950,4,1,92.0,65.0,...,6,0.294118,49.0,14.0,14.0,False,0.0,28.571429,0,0.0
1,1,3,0,Yugoslavia,Switzerland,1950,3,1,34.0,14.0,...,0.25,0.000000,27.0,9.0,8.0,False,0.0,33.333333,0,0.0
3,3,4,1,Yugoslavia,Mexico,1950,3,1,34.0,14.0,...,0.25,0.000000,49.0,14.0,14.0,False,0.0,28.571429,0,0.0
4,4,2,0,Brazil,Yugoslavia,1950,2,1,92.0,65.0,...,6,0.294118,34.0,14.0,8.0,2,0.0,41.176471,0.25,0.0
5,5,2,1,Switzerland,Mexico,1950,1,1,27.0,9.0,...,0,0.000000,49.0,14.0,14.0,False,0.0,28.571429,0,0.0


In [51]:
matches.to_csv('./src/matches.csv')

In [50]:
matcheswd = get_matches(with_team_stats=True,duplicate_with_reversed=True,exclude_ties=False)
        
matcheswd.head()

,id,score1,score2,team1,team2,year,score_diff,winner,matches_played,matches_won,...,podium_score_yearly,cups_won_yearly,matches_played_2,matches_won_2,years_played_2,podium_score_2,cups_won_2,matches_won_percent_2,podium_score_yearly_2,cups_won_yearly_2
0,0,4,0,Brazil,Mexico,1950,4,1,92.0,65.0,...,6,0.294118,49.0,14.0,14.0,False,0.0,28.571429,0,0.0
1,1,3,0,Yugoslavia,Switzerland,1950,3,1,34.0,14.0,...,0.25,0.000000,27.0,9.0,8.0,False,0.0,33.333333,0,0.0
2,2,2,2,Brazil,Switzerland,1950,0,0,92.0,65.0,...,6,0.294118,27.0,9.0,8.0,False,0.0,33.333333,0,0.0
3,3,4,1,Yugoslavia,Mexico,1950,3,1,34.0,14.0,...,0.25,0.000000,49.0,14.0,14.0,False,0.0,28.571429,0,0.0
4,4,2,0,Brazil,Yugoslavia,1950,2,1,92.0,65.0,...,6,0.294118,34.0,14.0,8.0,2,0.0,41.176471,0.25,0.0


In [52]:
matcheswd.to_csv('./src/matches.wd.csv')